In [1]:
import pandas as pd
import spacy
import torch


df = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)

# sample d'un très petit échantillon pour cet ordi
df = df.sample(1000)

In [2]:
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 

In [3]:
df["dupyo"] = df["Contribution"].apply(join_breaklines)

In [4]:
df["clean"] = df["dupyo"].apply(join_breaklinesn)

In [5]:
pd.set_option('display.max_colwidth', None)
df["clean"].head

<bound method NDFrame.head of 40565                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [6]:
documents = df.clean.tolist()
documents

['4. Démocratie et citoyenneté - Interdire le cumul de mandats. - Interdire d’occuper plus de deux fois un mandat électif, - Interdire les emplois fictifs ou de complaisances financés par les finances publiques, - Appliquer à l’état les mêmes règles que celles des pays Scandinaves, Mettre en place des lieux et moments pour que les élus rendent compte de leur action auprès des citoyens, - Accueillir décemment des migrants voulant résider en France, aider les migrants désirant se rendre au Royaume Uni, - Mettre en place le RIC. - Interdire de pratiquer une religion sur l’espace public , interdire de véhiculer toute religion basée sur l’intolérance, la haine et l’inégalité hommes femmes et démantèlement des structures créées dans cet objectif. Cette liste n’est bien sûr pas limitative et les solutions pour un monde meilleur et fraternel existent Notre pacte social doit reposer sur la justice fiscale, la solidarité, le respect et le dialogue, pour que chaque français puisse vivre dignement

In [7]:
# from sklearn.datasets import fetch_20newsgroups

# dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
# documents = dataset.data

In [8]:
type(documents)

list

In [9]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]

def remove_stopwords(text):
    stopwords = final_stopwords_list
    text = [word for word in text if word not in stopwords]
    return text

In [10]:
nostopword = remove_stopwords(documents)

In [11]:
def lemmatize(docs, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
  '''
  Performs lemmization of input documents.
  Args:
    - docs: list of strings with input documents
    - allowed_postags: list of accepted Part of Speech (POS) types
  Output:
    - list of strings with lemmatized input
  '''
  nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
  nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
  lemmatized_docs = []
  for doc in docs:
    doc = nlp(doc)
    tokens = []
    for token in doc:
      if token.pos_ in allowed_postags:
        tokens.append(token.lemma_)
    lemmatized_docs.append(" ".join(tokens))
  return (lemmatized_docs)

In [12]:
lemmatized = lemmatize(documents)

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 3

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words="english")
tf = tf_vectorizer.fit_transform(lemmatized)

In [15]:
# Créer le modèle LDA
lda = LatentDirichletAllocation(
        n_components=n_topics, 
        max_iter=5, 
        learning_method='online', 
        learning_offset=50.,
        random_state=0)

# Fitter sur les données
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=3, random_state=0)

In [16]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_vectorizer.get_feature_names_out(), 10)

Topic 0:
plus public pouvoir devoir falloir tout citoyen écologique service faire
Topic 1:
cahier monsieur doléance grand débat mairie national rue maire pas
Topic 2:
ne plus pas tout avoir pouvoir devoir faire retraite illisible


In [20]:
import pyLDAvis
import pyLDAvis.gensim_models


In [22]:
import collections

In [24]:
# Computes the frequency of all word forms in the train set.
word_counts = collections.defaultdict(int)
# keys will be given, key added if not existant, values will be int
for tokens in lemmatized:
  for token in tokens: word_counts[token] += 1

print(len(word_counts))

103


In [25]:
count_threshold = 1;
vocabulary = set()
for word, count in word_counts.items():
    if(count > count_threshold): vocabulary.add(word)

# print(vocabulary)
print(len(vocabulary))

94
